<a href="https://colab.research.google.com/github/stuart-lane/MachineLearning/blob/main/autograd2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import subprocess

# Set environment variables
os.environ["CUDA"] = "cpu"  # Set CUDA to CPU (no GPU support)
os.environ["TORCH_INSTALL"] = "1"

# Install the torch package (assuming you are using pip)
subprocess.run(["pip", "install", "torch"])

# Download and install PyTorch (assuming you want the CPU version)
subprocess.run(["pip", "install", "torch-cpu"])

CompletedProcess(args=['pip', 'install', 'torch-cpu'], returncode=1)

In [ ]:
pip install ISLP

In [2]:
from ISLP import load_data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
default = load_data("Default")

In [4]:
default['default1'] = (default['default'] == "Yes").astype(int)

In [5]:
from sklearn.model_selection import train_test_split

x = default[['balance', 'income']].values # Include a column of ones for the intercept
x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)  # Add a column of ones for the intercept
y = default['default1'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 123)

In [6]:
train_df_keys = {'balance': x_train[:,1], 'income': x_train[:,2], 'default': y_train }

train_df = pd.DataFrame(train_df_keys)

In [7]:
import torch
from torch.optim import LBFGS

In [23]:
# Assuming you have 'train' DataFrame with 'balance' and 'income' columns
x = torch.cat((torch.ones(train_df.shape[0], 1), torch.tensor(train_df[["balance", "income"]].values , dtype=torch.float64)), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["default"].values, dtype = torch.float64)
y.requires_grad = True
y = y.view(-1, 1)

beta = torch.randn(3, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

Running on: cpu


In [42]:
# Define the loss function

def ll(beta):
    xbetahat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbetahat)
    loss = - torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

optimizer = LBFGS([beta], lr = 0.001)

def calculate_loss():
    optimizer.zero_grad()
    value = ll(beta)
    value.backward()
    torch.nn.utils.clip_grad_value_(beta, 20)
    return value

In [44]:
for i in range(1, 10000):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  if (i % 1000) == 0:
    print(f"Iteration {i + 1} || Log-value {logl_value}")
    print(f"Estimated weights: {estimated_weights}")

Iteration 1001 || Log-value 909.2369482878473
Estimated weights: [[-6.16510734e-05]
 [ 1.84484509e-04]
 [-1.13211186e-04]]
Iteration 2001 || Log-value 909.225095284568
Estimated weights: [[-6.16585715e-05]
 [ 1.84920113e-04]
 [-1.13216982e-04]]
Iteration 3001 || Log-value 909.2132772003392
Estimated weights: [[-6.16660695e-05]
 [ 1.85355716e-04]
 [-1.13222778e-04]]
Iteration 4001 || Log-value 909.2014940468716
Estimated weights: [[-6.16735676e-05]
 [ 1.85791320e-04]
 [-1.13228574e-04]]


KeyboardInterrupt: ignored

In [11]:
# Define the initial values
debugstart = torch.tensor([1, 2, 3], dtype=torch.float64, requires_grad=True)
x0 = torch.tensor([1, 1/2, 1/3], dtype=torch.float64, requires_grad=True)
y1 = torch.tensor([1], dtype=torch.float64, requires_grad=True)

# Calculate xbhat
xbhat = torch.mm(x0.unsqueeze(0), debugstart.unsqueeze(1))
xbhat.retain_grad()

# Calculate yhat
yhat = torch.sigmoid(xbhat)
yhat.retain_grad()

# Calculate the loss
loss = -torch.sum(y1 * torch.log(yhat) + (1 - y1) * torch.log(1 - yhat))
loss.backward()

# Print the tensors and their gradients
print("y:", y1)
print("x:", x0)
print("xbhat:", xbhat)
print("Gradients of xbhat (dloss/dp):", xbhat.grad)
print("yhat:", yhat)
print("Gradients of yhat (dloss/dyhat):", yhat.grad)
print("Loss:", loss.item())

y: tensor([1.], dtype=torch.float64, requires_grad=True)
x: tensor([1.0000, 0.5000, 0.3333], dtype=torch.float64, requires_grad=True)
xbhat: tensor([[3.]], dtype=torch.float64, grad_fn=<MmBackward0>)
Gradients of xbhat (dloss/dp): tensor([[-0.0474]], dtype=torch.float64)
yhat: tensor([[0.9526]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
Gradients of yhat (dloss/dyhat): tensor([[-1.0498]], dtype=torch.float64)
Loss: 0.04858735157374191


TRY AGAIN

In [26]:
import os
import subprocess

# Set environment variables
os.environ["CUDA"] = "cpu"
os.environ["TORCH_INSTALL"] = "1"

# Install PyTorch
subprocess.run(["pip", "install", "torch"])

CompletedProcess(args=['pip', 'install', 'torch'], returncode=0)

In [27]:
from ISLP import load_data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
default = load_data("Default")
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [30]:
default['default1'] = (default['default'] == "Yes").astype(int)

In [31]:
from sklearn.model_selection import train_test_split

x = default[['balance', 'income']].values # Include a column of ones for the intercept
x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)  # Add a column of ones for the intercept
y = default['default1'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 123)

In [33]:
# Assuming you have 'train' DataFrame with 'balance' and 'income' columns
x = torch.cat((torch.ones(train_df.shape[0], 1), torch.tensor(train_df[["balance", "income"]].values , dtype=torch.float64)), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["default"].values, dtype = torch.float64)
y.requires_grad = True
y = y.view(-1, 1)

beta = torch.randn(3, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

print([x.shape, y.shape, beta.shape])

Running on: cpu
[torch.Size([5000, 3]), torch.Size([5000, 1]), torch.Size([3, 1])]


In [40]:
# Initializing beta as a parameter with requires_grad=True in PyTorch
beta = torch.zeros(3, 1, dtype=torch.float64, requires_grad=True)

# Define loss function: likelihood (binary cross-entropy in ML context)
def ll(beta):
    xbhat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbhat)
    loss = -torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

# Using LBFGS optimizer
optimizer = LBFGS([beta], lr=0.001)

# A 'closure' for each step of the optimizer
def calc_loss():
    def closure():
        optimizer.zero_grad()
        value = ll(beta)
        value.backward()
        return value
    return closure

In [46]:
from IPython.display import display, HTML

# Assuming optimizer and calc_loss functions are defined as per the previous Python translation

# Run optimization loop for 100 iterations
for i in range(1, 1000):
    logl = optimizer.step(calc_loss())  # 20 internal iterations per step
    logl_value = float(logl.detach().numpy())
    if (i % 100) == 0:
      display(HTML("<tt>{0}: {1}</tt>".format(i, logl_value)))

In [47]:
beta_detach = beta.detach().numpy()

In [48]:
print(beta_detach)

[[-1.11733541e+01]
 [ 5.55262641e-03]
 [ 1.68420993e-05]]
